In [2]:
pip install pyspark==3.0.0

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession

working_directory = 'jars/*'

spark = SparkSession.\
        builder.\
        appName("pyspark-with-mongo").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1g").\
        config("spark.mongodb.input.uri","mongodb://mongodb:27017/datascience.movies").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        config('spark.driver.extraClassPath', working_directory). \
        getOrCreate()

sc = spark.sparkContext

In [4]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource")\
    .option("uri", "mongodb://mongodb:27017/datascience.movies")\
    .load()

## Który aktor wystąpił najwięcej razy

In [6]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- actors: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- awards: struct (nullable = true)
 |    |-- wins: integer (nullable = true)
 |    |-- nominations: integer (nullable = true)
 |    |-- text: string (nullable = true)
 |-- countries: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- director: string (nullable = true)
 |-- genres: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- imdb: struct (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- rating: double (nullable = true)
 |    |-- votes: integer (nullable = true)
 |-- metacritic: integer (nullable = true)
 |-- plot: string (nullable = true)
 |-- poster: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released: timestamp (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- tomato: stru

In [9]:
from pyspark.sql.functions import *

In [24]:
Most_popular_actors = df.select('_id', explode_outer('actors').alias('actor'))\
    .where(column('actor').isNotNull())\
    .groupBy('actor')\
    .agg(count("actor").alias('movies_count'))\
    .orderBy(desc('movies_count'))\
    .limit(10)

In [25]:
Most_popular_actors.cache()

DataFrame[actor: string, movies_count: bigint]

In [26]:
(df.write.format("orc")
    .save("most_popular_actors.orc"))

In [27]:
sc.stop()